In [396]:
!pip install sympy
import sympy as sp
from sympy.physics.quantum import TensorProduct as TP
from IPython.display import display

Defaulting to user installation because normal site-packages is not writeable


In [397]:
id = sp.symbols("𝟙", commutative=False)
n_op = sp.symbols("n", commutative=False)

In [398]:
def gate_at(d: dict[int, sp.Symbol], n: int):
    """Utility function for filling operators with identities."""
    return TP(*(
        id if i not in d
        else d[i]
        for i in range(n)
    ))

#### Implementation of the reference hamiltonian as a sum of terms.

The $A_i$ are the single-qubit terms (2*2 matrices):

$$
A_i = \frac{\hbar}{2} \left( \Omega_i(t) \sigma^x - \delta_i(t) \sigma^z \right)
$$

The $U_{ij}$ are the interaction matrix coefficients.

In [399]:
def A(i: int):
    return sp.Symbol(f"A_{i}", commutative=False)

def U(i: int, j: int):
    return sp.Symbol(f"U_{i}{j}")

In [400]:
def reference_hamiltonian(qubit_count: int):
    result = sum(gate_at({i: A(i)}, qubit_count) for i in range(qubit_count))

    result += sum(
        U(j, i) *
        gate_at({
            i: n_op,
            j: n_op
        }, qubit_count)
        for i in range(qubit_count) for j in range(i)
    )

    return result

#### Definition of MPO factor multiplication.

MPO factors are seen as matrices containing 2*2 matrices (in the case of qubits).

These matrices of matrices are multiplied by using the tensor product for element-wise multiplication.


In [401]:
def mpo_factors_product_pairwise(a: sp.Matrix, b: sp.Matrix):
    """Matrix product where element-wise multiplication is the tensor product."""

    assert sp.shape(a)[1] == sp.shape(b)[0], "Incompatible matrix dimensions"

    common_dim = sp.shape(a)[1]

    res_rows = sp.shape(a)[0]
    res_cols = sp.shape(b)[1]

    res = sp.Matrix([
            [sum(TP(a[row, k], b[k, col]).expand(tensorproduct=True) for k in range(common_dim))
                for col in range(res_cols)
            ]
        for row in range(res_rows)
    ])

    if res_rows == res_cols == 1:
        return res[0, 0]

    return res


def mpo_factors_product(*args):
    """n-ary matrix product where element-wise multiplication is the tensor product."""

    assert len(args) >= 2
    if len(args) == 2:
        return mpo_factors_product_pairwise(*args)
    
    return mpo_factors_product_pairwise(args[0], mpo_factors_product(*args[1:]))

#### The below is an example of Ising Hamiltonian MPO factors implementation.

In [402]:
def get_extreme_factor(i):
    return sp.Matrix([[A(i), id, n_op, 0]])

In [403]:
def get_factor(i, inter_to = 0):
    n = abs(i - inter_to)

    inters = [U(j, i) for j in range(i)] if inter_to == 0 else [U(i, j) for j in range(inter_to, i, -1)]

    assert n >= 1
    rows = n + 3
    cols = n + 4
    # Expressed col by col, then transposed.
    return sp.Matrix([
        [id, A(i)] + [0] * (rows - 2),
        [0, id] + [0] * (rows - 2),
        *([0, 0] + [id if x == c else 0 for x in range(rows - 2)]
          for c in range(cols - 4)
        ),
        [0, n_op] + [0] * (rows - 2),
        [0, 0] + [inter * n_op for inter in inters] + [id]
    ]).T

In [404]:
def get_middle_factor(where, qubit_count):
    return sp.Matrix([
        [0, id, *(0 for _ in range(qubit_count - where))],
        [id, A(where), *(U(where, i) * n_op for i in range(qubit_count - 1, where, -1)), id],
        *([0, U(i, where) * n_op, *(U(i, j) * id for j in range(qubit_count - 1, where, -1)), 0] for i in range(where)),
        [0, id, *(0 for _ in range(qubit_count - where))],
    ])

In [405]:
def get_ising_mpo_factors(qubit_count: int):
    """Returns all the Ising MPO factors for all qubits."""
    middle = qubit_count // 2
    return ([get_extreme_factor(0)] +
            [get_factor(i, inter_to=0) for i in range(1, middle)]
            + [get_middle_factor(middle, qubit_count=qubit_count)] +
            [get_factor(i, inter_to=qubit_count - 1).T for i in range(middle + 1, qubit_count - 1)]
            + [get_extreme_factor(qubit_count - 1).T]
        )

#### Testing by comparing the product of the MPO factors and the reference Hamiltonian

In [406]:
for factor in get_ising_mpo_factors(10):
    display(factor)

Matrix([[A_0, 𝟙, n, 0]])

Matrix([
[  𝟙, 0, 0, 0,      0],
[A_1, 𝟙, 0, n,      0],
[  0, 0, 𝟙, 0, U_01*n],
[  0, 0, 0, 0,      𝟙]])

Matrix([
[  𝟙, 0, 0, 0, 0,      0],
[A_2, 𝟙, 0, 0, n,      0],
[  0, 0, 𝟙, 0, 0, U_02*n],
[  0, 0, 0, 𝟙, 0, U_12*n],
[  0, 0, 0, 0, 0,      𝟙]])

Matrix([
[  𝟙, 0, 0, 0, 0, 0,      0],
[A_3, 𝟙, 0, 0, 0, n,      0],
[  0, 0, 𝟙, 0, 0, 0, U_03*n],
[  0, 0, 0, 𝟙, 0, 0, U_13*n],
[  0, 0, 0, 0, 𝟙, 0, U_23*n],
[  0, 0, 0, 0, 0, 0,      𝟙]])

Matrix([
[  𝟙, 0, 0, 0, 0, 0, 0,      0],
[A_4, 𝟙, 0, 0, 0, 0, n,      0],
[  0, 0, 𝟙, 0, 0, 0, 0, U_04*n],
[  0, 0, 0, 𝟙, 0, 0, 0, U_14*n],
[  0, 0, 0, 0, 𝟙, 0, 0, U_24*n],
[  0, 0, 0, 0, 0, 𝟙, 0, U_34*n],
[  0, 0, 0, 0, 0, 0, 0,      𝟙]])

Matrix([
[0,      𝟙,      0,      0,      0,      0, 0],
[𝟙,    A_5, U_59*n, U_58*n, U_57*n, U_56*n, 𝟙],
[0, U_05*n, U_09*𝟙, U_08*𝟙, U_07*𝟙, U_06*𝟙, 0],
[0, U_15*n, U_19*𝟙, U_18*𝟙, U_17*𝟙, U_16*𝟙, 0],
[0, U_25*n, U_29*𝟙, U_28*𝟙, U_27*𝟙, U_26*𝟙, 0],
[0, U_35*n, U_39*𝟙, U_38*𝟙, U_37*𝟙, U_36*𝟙, 0],
[0, U_45*n, U_49*𝟙, U_48*𝟙, U_47*𝟙, U_46*𝟙, 0],
[0,      𝟙,      0,      0,      0,      0, 0]])

Matrix([
[𝟙, A_6,      0,      0,      0, 0],
[0,   𝟙,      0,      0,      0, 0],
[0,   0,      𝟙,      0,      0, 0],
[0,   0,      0,      𝟙,      0, 0],
[0,   0,      0,      0,      𝟙, 0],
[0,   n,      0,      0,      0, 0],
[0,   0, U_69*n, U_68*n, U_67*n, 𝟙]])

Matrix([
[𝟙, A_7,      0,      0, 0],
[0,   𝟙,      0,      0, 0],
[0,   0,      𝟙,      0, 0],
[0,   0,      0,      𝟙, 0],
[0,   n,      0,      0, 0],
[0,   0, U_79*n, U_78*n, 𝟙]])

Matrix([
[𝟙, A_8,      0, 0],
[0,   𝟙,      0, 0],
[0,   0,      𝟙, 0],
[0,   n,      0, 0],
[0,   0, U_89*n, 𝟙]])

Matrix([
[A_9],
[  𝟙],
[  n],
[  0]])

In [407]:
mpo_factors_product(*get_ising_mpo_factors(10))

U_01*nxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_02*nx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_03*nx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_04*nx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_05*nx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_06*nx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_07*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_08*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_09*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_12*𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_13*𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_14*𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_15*𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_16*𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_17*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_18*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_19*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_23*𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_24*𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_25*𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_26*𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_27*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_28*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_29*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_34*𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_35*𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_36*𝟙x𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_37*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_38*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_39*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xn + U_45*𝟙x𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙 + U_46*𝟙x𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙 + U_47*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙 + U_48*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙 + U_49*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xn + U_56*𝟙x𝟙x𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙 + U_57*𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙 + U

In [408]:
reference_hamiltonian(10)

U_01*nxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_02*nx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_03*nx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_04*nx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_05*nx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_06*nx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_07*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_08*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_09*nx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_12*𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_13*𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_14*𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_15*𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_16*𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_17*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_18*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_19*𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_23*𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙 + U_24*𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_25*𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_26*𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_27*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_28*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_29*𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙x𝟙xn + U_34*𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙x𝟙 + U_35*𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙x𝟙 + U_36*𝟙x𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙x𝟙 + U_37*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙x𝟙 + U_38*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xnx𝟙 + U_39*𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙x𝟙xn + U_45*𝟙x𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙x𝟙 + U_46*𝟙x𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙x𝟙 + U_47*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙xnx𝟙x𝟙 + U_48*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙xnx𝟙 + U_49*𝟙x𝟙x𝟙x𝟙xnx𝟙x𝟙x𝟙x𝟙xn + U_56*𝟙x𝟙x𝟙x𝟙x𝟙xnxnx𝟙x𝟙x𝟙 + U_57*𝟙x𝟙x𝟙x𝟙x𝟙xnx𝟙xnx𝟙x𝟙 + U

In [409]:
str(reference_hamiltonian(10)) == str(mpo_factors_product(*get_ising_mpo_factors(10)))

True